In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
import tensorflow_datasets as tfds

# 9.

In [ ]:
train_set, valid_set, test_set = tfds.load(
    'mnist',
    split=['train[:50000]', 'train[50000:]', 'test'],
    as_supervised=True,
    shuffle_files=True,
)

In [7]:
class RBFLayer(layers.Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [5]:
def normalize(image):
    return tf.cast(image, tf.float32) / 255.


model = keras.models.Sequential([
    layers.Input((28, 28, 1)),
    layers.Lambda(normalize),
    layers.ZeroPadding2D(padding=2),
    layers.Conv2D(filters=6, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=[2, 2]),
    layers.Conv2D(filters=16, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=[2, 2]),
    layers.Conv2D(filters=120, kernel_size=5, activation='tanh'),
    layers.Flatten(),
    layers.Dense(84, activation='tanh'),
    RBFLayer(10, 0.5)
])
model.compile(
    optimizer='nadam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
batch_size = 128
train_set = train_set.batch(batch_size).prefetch(1)
test_set = test_set.batch(batch_size).prefetch(1)
valid_set = valid_set.batch(batch_size).prefetch(1)

In [10]:
model.fit(
    train_set,
    epochs=10,
    validation_data=valid_set
)
model.evaluate(test_set)

Epoch 1/10
391/391 [==============================] - 9s 15ms/step - loss: 0.4904 - accuracy: 0.8854 - val_loss: 0.1457 - val_accuracy: 0.9587
Epoch 2/10
391/391 [==============================] - 3s 7ms/step - loss: 0.1089 - accuracy: 0.9694 - val_loss: 0.0850 - val_accuracy: 0.9762
Epoch 3/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0695 - accuracy: 0.9803 - val_loss: 0.0718 - val_accuracy: 0.9787
Epoch 4/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0527 - accuracy: 0.9854 - val_loss: 0.0651 - val_accuracy: 0.9805
Epoch 5/10
391/391 [==============================] - 3s 6ms/step - loss: 0.0424 - accuracy: 0.9881 - val_loss: 0.0602 - val_accuracy: 0.9810
Epoch 6/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0346 - accuracy: 0.9907 - val_loss: 0.0568 - val_accuracy: 0.9823
Epoch 7/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0287 - accuracy: 0.9925 - val_loss: 0.0544 - val_accuracy: 0.9838
Epoch

[0.04711465537548065, 0.9868000149726868]

# 10.

In [12]:
train_set, valid_set, test_set = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    shuffle_files=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.PPQECD_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.PPQECD_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.
1020
1020
6149


In [ ]:
model = keras.applications.VGG19(weights='imagenet')